<a href="https://colab.research.google.com/github/alvaradospc/IBM_Clases/blob/master/PY0101EN_5_2_API_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://cognitiveclass.ai/">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/Ad/CCLog.png" width="200" align="center">
</a>

<h1 align="center"><font size="5"><b>Application Programming Interface</b></font></h1>

<p>In this notebook, you will learn to convert an audio file of an English speaker to text using a Speech to Text API. Then you will translate the English version to a Spanish version using a Language Translator API. <b>Note:</b> You must obtain the API keys and enpoints to complete the lab.</p>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <a href="https://cocl.us/topNotebooksPython101Coursera">
         <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/Ad/TopAd.png" width="750" align="center">
    </a>
</div>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>Table of Contents</h2>
<ul>
    <li><a href="#ref0">Speech To Text</a></li>
    <li><a href="#ref1">Language Translator</a></li>
    <li><a href="#ref2">Exercise</a></li>
</ul>
<br>
<p>Estimated Time Needed: <strong>25 min</strong></p>
</div>

In [1]:
#you will need the following library 
!pip install ibm_watson wget

     |████████████████████████████████| 368kB 2.8MB/s 
     |████████████████████████████████| 204kB 47.4MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-4.3.0-cp36-none-any.whl size=354170 sha256=a6db3ee5d6aa8d4cc8774eb3cdc0b2f9be42fb1998b04ec4d42d4cb1cab29e1a
  Stored in directory: /root/.cache/pip/wheels/b7/10/b4/509307276e9d558a6b06d8e69f8faf6cfbdf2d1b83abb6b8d4
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=06d520855116884352ba647b7641c8edf176699afc070272b908948b7907e90b
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.5.1-cp36-none-any.whl size=44491 sha256=a1339f6d2234fde813d0bf72dc230df852c2ec789e8c364d6e3517ebba9d6841
  Stored in directory: /root/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson wget ibm-cloud-sdk-core


<h2 id="ref0">Speech to Text</h2>

<p>First we import <code>SpeechToTextV1</code> from <code>ibm_watson</code>.For more information on the API, please click on this <a href="https://cloud.ibm.com/apidocs/speech-to-text?code=python">link</a></p>

In [0]:
from ibm_watson import SpeechToTextV1 
import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

<p>The service endpoint is based on the location of the service instance, we store the information in the variable URL. To find out which URL to use, view the service credentials.</p>

In [0]:
url_s2t = "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/70472748-f987-4316-8415-ba3601c8b396"

<p>You require an API key, and you can obtain the key on the <a href="https://cloud.ibm.com/resources">Dashboard </a>.</p>

In [0]:
iam_apikey_s2t = "h2T0h6gVNomhxKJZeQua_jRLiSzq_TMQ5kdOsKNEJhJz"

<p>You create a <a href="http://watson-developer-cloud.github.io/python-sdk/v0.25.0/apis/watson_developer_cloud.speech_to_text_v1.html">Speech To Text Adapter object</a> the parameters are the  endpoint and API key.</p>

In [5]:
authenticator = IAMAuthenticator(iam_apikey_s2t)
s2t = SpeechToTextV1(authenticator=authenticator)
s2t.set_service_url(url_s2t)
s2t

<p>Lets download the audio file that we will use to convert into text.</p>

In [6]:
!wget -O PolynomialRegressionandPipelines.mp3  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/PolynomialRegressionandPipelines.mp3



--2020-03-31 19:39:00--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/PolynomialRegressionandPipelines.mp3
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4234179 (4.0M) [audio/mpeg]
Saving to: ‘PolynomialRegressionandPipelines.mp3’

PolynomialRegressio 100%[===================>]   4.04M  --.-KB/s    in 0.1s    

2020-03-31 19:39:00 (36.0 MB/s) - ‘PolynomialRegressionandPipelines.mp3’ saved [4234179/4234179]



<p>We have the path of the wav file we would like to convert to text</p>

In [0]:
filename='PolynomialRegressionandPipelines.mp3'

<p>We create the file object <code>wav</code> with the wav file using  <code>open</code> ; we set the <code>mode</code> to  "rb" ,  this is similar to read mode, but it ensures the file is in binary mode.We use the method <code>recognize</code> to return the recognized text. The parameter audio is the file object <code>wav</code>, the parameter <code>content_type</code> is the format of the audio file.</p>

In [0]:
with open('/content/PolynomialRegressionandPipelines.mp3', mode="rb")  as wav:
    response = s2t.recognize(audio=wav, content_type='audio/mp3')

<p>The attribute result contains a dictionary that includes the translation:</p>

In [11]:
response.result

{'result_index': 0,
 'results': [{'alternatives': [{'confidence': 0.94,
     'transcript': 'in this video we will cover polynomial regression and pipelines '}],
   'final': True},
  {'alternatives': [{'confidence': 0.9,
     'transcript': "what do we do when a linear model is not the best fit for our data let's look into another type of regression model the polynomial regression we transform our data into a polynomial then use linear regression to fit the parameters that we will discuss pipelines pipelines are way to simplify your code "}],
   'final': True},
  {'alternatives': [{'confidence': 0.95,
     'transcript': "polynomial regression is a special case of the general linear regression this method is beneficial for describing curvilinear relationships what is a curvilinear relationship it's what you get by squaring or setting higher order terms of the predictor variables in the model transforming the data the model can be quadratic which means the predictor variable in the model i

In [12]:
from pandas.io.json import json_normalize

json_normalize(response.result['results'],"alternatives")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,confidence,transcript
0,0.94,in this video we will cover polynomial regress...
1,0.90,what do we do when a linear model is not the b...
2,0.95,polynomial regression is a special case of the...
3,0.95,the model can be cubic which means the predict...
4,0.91,there also exists higher order polynomial regr...
5,0.89,let's look at an example from our data we gene...
6,0.92,in python we do this by using the poly fit fun...
7,0.90,negative one point five five seven X. one cute...
8,0.90,consider the feature shown here applying the m...
9,0.89,pipeline sequentially perform a series of tran...


In [13]:
response

<p>We can obtain the recognized text and assign it to the variable <code>recognized_text</code>:</p>

In [14]:
recognized_text=response.result['results'][0]["alternatives"][0]["transcript"]
type(recognized_text)

str

<h2 id="ref1">Language Translator</h2>

<p>First we import <code>LanguageTranslatorV3</code> from ibm_watson. For more information on the API click <a href="https://cloud.ibm.com/apidocs/speech-to-text?code=python"> here</a></p>

In [0]:
from ibm_watson import LanguageTranslatorV3

<p>The service endpoint is based on the location of the service instance, we store the information in the variable URL. To find out which URL to use, view the service credentials.</p>

In [0]:
url_lt='https://api.us-south.language-translator.watson.cloud.ibm.com/instances/c304032c-d0c2-4ae1-a388-b367c6af2883'

<p>You require an API key, and you can obtain the key on the <a href="https://cloud.ibm.com/resources">Dashboard</a>.</p>

In [0]:
apikey_lt='VGsbSEtf3YcUfXY5IFwzZGjEB6w6sdb97Ftfo1iv3ql2'

<p>API requests require a version parameter that takes a date in the format version=YYYY-MM-DD. This lab describes the current version of Language Translator, 2018-05-01</p>

In [0]:
version_lt='2018-05-01'

<p>we create a  Language Translator object <code>language_translator</code>:</p>

In [19]:
authenticator = IAMAuthenticator(apikey_lt)
language_translator = LanguageTranslatorV3(version=version_lt,authenticator=authenticator)
language_translator.set_service_url(url_lt)
language_translator

<p>We can get a Lists the languages that the service can identify.
The method Returns the language code.  For example English (en) to  Spanis (es) and name of each language.</p>

In [20]:
from pandas.io.json import json_normalize

json_normalize(language_translator.list_identifiable_languages().get_result(), "languages")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,language,name
0,af,Afrikaans
1,ar,Arabic
2,az,Azerbaijani
3,ba,Bashkir
4,be,Belarusian
...,...,...
63,uk,Ukrainian
64,ur,Urdu
65,vi,Vietnamese
66,zh,Simplified Chinese


<p>We can use the method <code>translate</code> this will translate the text. The parameter text is the text. Model_id is the type of model we would like to use use we use list the the langwich . In this case, we set it to 'en-es' or English to Spanish. We get a Detailed Response object translation_response</p>

In [21]:
translation_response = language_translator.translate(\
    text=recognized_text, model_id='en-es')
translation_response

<p>The result is a dictionary.</p>

In [22]:
translation=translation_response.get_result()
translation

{'character_count': 64,
 'translations': [{'translation': 'en este video cubriremos la regresión polinómica y los oleoductos '}],
 'word_count': 10}

<p>We can obtain the actual translation as a string as follows:</p>

In [23]:
spanish_translation =translation['translations'][0]['translation']
spanish_translation 

'en este video cubriremos la regresión polinómica y los oleoductos '

<p>We can translate back to English</p>

In [0]:
translation_new = language_translator.translate(text=spanish_translation ,model_id='es-en').get_result()

<p>We can obtain the actual translation as a string as follows:</p>

In [25]:
translation_eng=translation_new['translations'][0]['translation']
translation_eng

'in this video, we will cover the polynomial regression and the pipelines '

<p>We can convert it to french as well:</p>

In [0]:
French_translation=language_translator.translate(
    text=translation_eng , model_id='en-fr').get_result()

In [27]:
French_translation['translations'][0]['translation']

'Dans cette vidéo, nous couvrons la régression polynomiale et les pipelines '

<h3>Language Translator</h3>

 <a href="http://cocl.us/NotebooksPython101bottom"><img src="https://ibm.box.com/shared/static/irypdxea2q4th88zu1o1tsd06dya10go.png" width="750" align="center"></a>

<b>References</b>

https://cloud.ibm.com/apidocs/speech-to-text?code=python

https://cloud.ibm.com/apidocs/language-translator?code=python

<hr>

<h4>About the Author:</h4>
<p><a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.</p>

Other contributor: <a href="https://www.linkedin.com/in/fanjiang0619/">Fan Jiang</a>

Copyright &copy; 2019 [cognitiveclass.ai](https:cognitiveclass.ai). This notebook and its source code are released under the terms of the [MIT License](cognitiveclass.ai).